In [56]:
# Imports
from PIL import Image
import numpy as np
import cv2
import math
from scipy.fft import dct

In [6]:
#convert image to grey scale
im = cv2.imread('data/yasuo.jpeg')
im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

In [16]:
#getting an 8x8 block
init_x=100
init_y=100
block = im[init_x:init_x+8, init_y:init_y+8]

In [30]:
block

array([[43, 58, 53, 27, 24, 53, 47, 51],
       [47, 53, 43, 26, 34, 48, 38, 43],
       [53, 48, 34, 26, 35, 39, 38, 46],
       [56, 45, 28, 27, 31, 32, 43, 52],
       [51, 40, 36, 11, 39, 31, 47, 28],
       [46, 42, 33, 11, 37, 36, 38, 36],
       [39, 45, 29, 14, 34, 40, 28, 44],
       [33, 47, 26, 17, 33, 39, 26, 42]], dtype=uint8)

In [18]:
#define c function used in DCT
def c_funct(i):
    if i==0:
        return 1/(2**(1/2))
    else:
        return 1

In [44]:
def dct_matrix():
    result_matrix = np.zeros(shape=(8,8))
    sum = 0
    for i in range(8):
        for j in range(8):
            if i ==0:
                result_matrix[i,j] = np.sqrt(1/8)
            else:
                result_matrix[i,j] = np.sqrt(2/8)*math.cos(((2*j+1)*i*math.pi)/16)
    return result_matrix

In [80]:
np.round(dct_matrix(),3)

array([[ 0.354,  0.354,  0.354,  0.354,  0.354,  0.354,  0.354,  0.354],
       [ 0.49 ,  0.416,  0.278,  0.098, -0.098, -0.278, -0.416, -0.49 ],
       [ 0.462,  0.191, -0.191, -0.462, -0.462, -0.191,  0.191,  0.462],
       [ 0.416, -0.098, -0.49 , -0.278,  0.278,  0.49 ,  0.098, -0.416],
       [ 0.354, -0.354, -0.354,  0.354,  0.354, -0.354, -0.354,  0.354],
       [ 0.278, -0.49 ,  0.098,  0.416, -0.416, -0.098,  0.49 , -0.278],
       [ 0.191, -0.462,  0.462, -0.191, -0.191,  0.462, -0.462,  0.191],
       [ 0.098, -0.278,  0.416, -0.49 ,  0.49 , -0.416,  0.278, -0.098]])

In [82]:
np.round(np.matrix(dct_matrix().transpose())* np.matrix(block) * np.matrix(dct_matrix()),3)

array([[267.512, -53.063, 109.374,  -4.399,  26.564, -20.922,  20.473,
         15.102],
       [-44.418,  11.999, -23.409, -11.896, -13.66 ,  10.034,   4.433,
        -13.271],
       [ 55.329, -13.834,  17.124,  -2.409,  -7.455, -22.379,   9.995,
        -11.351],
       [-11.701,   3.226,  -4.537,  -0.445, -17.898,   6.197,  -5.235,
         11.324],
       [ 30.362,  -7.19 ,  14.079,  -5.806,  -1.363,  -2.093,   0.328,
         -1.958],
       [  2.787,  -1.321,   7.098,  -9.607,   3.675,   0.643,   6.465,
         -6.216],
       [ 19.614,  -4.66 ,  10.619,  -5.513,   1.653,  -1.931,   3.124,
         -3.262],
       [ 10.11 ,  -1.713,   4.09 ,  -1.239,  -5.571,  -0.537,  -0.477,
          3.406]])

In [72]:
np.rint((dct(block)))

array([[712.,   4.,  79., -21., -93., -25.,  35., -21.],
       [664.,  24.,  55.,  19., -45., -40.,  23.,  -8.],
       [638.,  21.,  80.,  28.,   1., -29.,   5.,   1.],
       [628.,   5., 114.,  18.,  25.,  -8., -13.,   1.],
       [566.,  28.,  69.,  62., -35.,  -5., -15.,  80.],
       [558.,  13.,  71.,  50., -27., -41.,   6.,  45.],
       [546.,  -2.,  68.,  29., -16., -76.,  19.,   0.],
       [526.,  -3.,  52.,  20., -18., -83.,   4., -17.]])

In [50]:
#computing DCT
dct_result = np.zeros(shape=(8,8))


In [29]:
np.rint(dct_result)

array([[302.,   8.,  52.,  18., -18., -27.,   6.,   7.],
       [ 30.,   4.,   4.,  -9.,  -9.,  12.,   4.,  -6.],
       [  4.,  -4.,  -7., -11., -13., -13.,   9., -16.],
       [  2.,   1.,  -1.,   2., -15.,   4.,   3.,  11.],
       [  2.,  -2.,   3.,  -4.,  -3.,   6.,  -4.,   0.],
       [  5.,  -5.,   8.,  -7.,   5.,  -0.,   2.,  -9.],
       [  0.,  -0.,   1.,  -1.,  -1.,   1.,  -0.,   1.],
       [ -2.,   2.,  -3.,   3.,  -4.,   0.,   0.,   6.]])